In [1]:
from scipy.io import wavfile
from scipy.signal import stft,get_window, correlate
import numpy as np
import pysepm as pm #import fwSNRseg,SNRseg,llr,wss, composite,pesq,stoi

# frequency weighted segmental SNR (fwSNRSeg)

In [2]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.fwSNRseg(cleanSig, enhancedSig, fs)# should be  3.3554


3.3554001065824384

In [3]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.fwSNRseg(cleanSig, enhancedSig, fs)# should be  10.0938


10.093763940018947

In [4]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.fwSNRseg(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be  12.5602


12.560192001043312

# Segmental SNR (SNRSeg)

In [5]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.SNRseg(cleanSig, enhancedSig, fs)# should be -4.0387


-4.0386645840708395

In [6]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.SNRseg(cleanSig, enhancedSig, fs)# should be 0.9595


0.959472693785315

In [7]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.SNRseg(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be 5.2814


5.281379775527017

# Log Likelihood Ratio (LLR)

In [8]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.llr(cleanSig, enhancedSig, fs)# should be 0.9593


0.9607521284185142

In [9]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.llr(cleanSig, enhancedSig, fs)# should be 0.6400


0.6399952859427331

In [10]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.llr(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be 0.2227

0.22273917406045848

# Weighted Spectral Slope (WSS)

In [11]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.wss(cleanSig, enhancedSig, fs)# should be 52.6579

52.65786610835284

In [12]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.wss(cleanSig, enhancedSig, fs)# should be 37.6490

37.649017195834844

In [13]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.wss(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be 19.6127

19.612652522957717

# PESQ

In [20]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.pesq(cleanSig, enhancedSig, fs)# should be 1.0832

1.0832337141036987

In [21]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.pesq(cleanSig, enhancedSig, fs)# should be 2.4634

2.463416557463939

In [22]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.pesq(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be 2.7293

2.7307331381405935

# Composite Objective Speech Quality Measure

In [14]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.composite(cleanSig, enhancedSig, fs)# should be (2.2837,1.5287,1.6055)

(2.283655194486704, 1.5287447837866348, 1.6054929873447288)

In [15]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.composite(cleanSig, enhancedSig, fs)# should be (3.5810,2.6084,2.9858)

(3.5810438801531697, 2.608416773805394, 2.985829621984948)

In [16]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.composite(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be (4.3330,3.1340,3.5397)

(4.333919599483947, 3.1347287982287013, 3.540909151423519)

# STOI

In [23]:
fs, cleanSig = wavfile.read("./samples/speech.wav")
fs, enhancedSig = wavfile.read("./samples/speech_bab_0dB.wav")
pm.stoi(cleanSig, enhancedSig, fs)# should be 0.6739

0.6739177895331303

In [24]:
fs, cleanSig = wavfile.read("./samples/sp04.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.stoi(cleanSig, enhancedSig, fs)# should be 0.8934

0.8934582975768947

In [25]:
fs, cleanSig = wavfile.read("./samples/enhanced.wav")
fs, enhancedSig = wavfile.read("./samples/sp04_babble_sn10.wav")
pm.stoi(cleanSig, enhancedSig[0:len(cleanSig)], fs) # should be 0.8917

0.8916166026077093